In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from dotenv import load_dotenv, find_dotenv
from smolagents import (
    CodeAgent,
    DuckDuckGoSearchTool,
    InferenceClientModel,
    VisitWebpageTool,
    WikipediaSearchTool,
    FinalAnswerTool,
    PythonInterpreterTool,
    Tool,
)
import sys
from pyprojroot import find_root, has_file
_ = load_dotenv(find_dotenv())

In [3]:
project_root_dir = find_root(has_file("README.md")).as_posix()
sys.path.extend([project_root_dir])


In [4]:
from huggingface_hub import model_info, InferenceClient

In [5]:
from PIL import Image
from io import BytesIO
import base64
from pathlib import Path

In [6]:
client = InferenceClient(
    provider="hyperbolic",
    bill_to="VitalNest",
)


In [2]:
all_inference_models = !curl -s https://huggingface.co/api/models | jq ".[].id"

In [ ]:
info = model_info("google/gemma-3-27b-it", expand="inference")
info.inference


'warm'

In [22]:
vlm_model = InferenceClientModel(model_id="Qwen/Qwen2.5-VL-72B-Instruct",bill_to="VitalNest",
                                 timeout=300,temperature=.1, max_tokens=10000)

In [ ]:
# def encode_image_to_base64(image_path):
#     """
#     Encodes an image file to a base64 string.

#     Args:
#         image_path (str): Path to the image file.

#     Returns:
#         str: Base64 encoded image string.
#     """
#     """Encodes a PIL image to a base64 string."""
#     image = Image.open(image_path)
#     buffered = BytesIO()
#     image.save(buffered, format="PNG")
#     return base64.b64encode(buffered.getvalue()).decode("utf-8")


In [ ]:
# # MAXIMUM of 4 images can be sent
# img_paths = [
#     Path(
#         "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_42.png"
#     ),
#     Path(
#         "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_12.png"
#     ),
#     Path(
#         "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_1.png"
#     ),
#     # Path(
#     #     "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_43.png"
#     # ),
#     # Path(
#     #     "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_44.png"
#     # ),
# ]

In [ ]:
# # grouped_images = [Image.open(fp=pth) for pth in img_paths]
# grouped_images = [encode_image_to_base64(pth) for pth in img_paths]

In [ ]:
# text_query = """what critical illnesses are covered under optima restore?
# If the answer isn't in these documents, say you don't know."""
# chat_template = [
#     {
# "role":"system", "content":"""You find answers from the relevant documents. Answer only 
# from these documents. If answer isn't available return 'Question cannot be answered based
# on the documents provided.' """
#     },
#     {
#         "role": "user",
#         "content": [
#             {
#                 "type": "image_url",
#                 "image_url": {"url": f"data:image/jpeg;base64,{image}"},
#             }
#             for image in grouped_images
#         ]
#         + [{"type": "text", "text": text_query}],
#     }
# ]


In [ ]:
# print(vlm_model.generate(messages=chat_template).content)

The critical illnesses covered under Optima Restore include Cancer, Open Chest CABG, First Heart Attack, Kidney Failure, Major Organ/Bone Marrow Transplant, Multiple Sclerosis, Permanent Paralysis of Limbs, and Stroke.


In [ ]:
# completion = client.chat.completions.create(
#     model="Qwen/Qwen2.5-VL-72B-Instruct",
#     messages=chat_template,
#     temperature=.1,
#     max_tokens=10_000
# )

# print(completion.choices[0].message.content)

# # completion = client.chat.completions.create(
# #     model="Qwen/Qwen2.5-VL-72B-Instruct",
# #     messages=[
# #         {
# #             "role": "user",
# #             "content": [
# #                 {"type": "text", "text": "Describe the images in one sentence."},
# #                 {
# #                     "type": "image_url",
# #                     "image_url": {
# #                         "url": "https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg"
# #                     },
# #                 },
# #                 {
# #                     "type": "image_url",
# #                     "image_url": {
# #                         "url": "https://upload.wikimedia.org/wikipedia/commons/3/3d/Nicu%C8%99or_Dan_%288_mai_2025%29.jpg"
# #                     },
# #                 },
# #             ],
# #         }
# #     ],
# # )

# # print(completion.choices[0].message)


The critical illnesses covered under Optima Restore include Cancer, Open Chest CABG, First Heart Attack, Kidney Failure, Major Organ/Bone Marrow Transplant, Multiple Sclerosis, Permanent Paralysis of Limbs, and Stroke.


In [8]:
class InsuranceInfoRetriever(Tool):
    name = "InsuranceInfoRetriever"
    description = "Retrieves information from insurance documents."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to search for."
        },
    }
    output_type = "string"

    def forward(self, query: str) -> str:
        client = InferenceClient(
            provider="hyperbolic",
            bill_to="VitalNest",
        )
        results = rag_app.search_documents(query)
        img_paths = [Path(res[0]) for res in results]

        grouped_images = [rag_app.encode_image_to_base64(pth) for pth in img_paths]
        chat_template = [
            {
        "role":"system", "content":"""You find answers from the relevant documents. Answer only 
        from these documents. If answer isn't available return 'Question cannot be answered based
        on the documents provided.' """
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{image}"},
                    }
                    for image in grouped_images
                ]
                + [{"type": "text", "text": query}],
            }
        ]
        completion = client.chat.completions.create(
            model="Qwen/Qwen2.5-VL-72B-Instruct",
            messages=chat_template,
            temperature=0.1,
            max_tokens=10_000,
        )
        answer = completion.choices[0].message.content
        return answer if answer else ""

# class SaveContent(Tool):
#     name = "SaveContent"
#     description = "Saves content to a file."
#     inputs = {
#         "content": {
#             "type": "string",
#             "description": "The content to save."
#         },
#         "filename": {
#             "type": "string",
#             "description": "The filename."
#         }
#     }
#     output_type = "string"

#     def forward(self, content: str, filename: str ):
#         with open(filename, "w") as f:
#             f.write(content)
#         return f"Content saved to {filename}"



In [9]:
insurance_agent = CodeAgent(
    tools=[InsuranceInfoRetriever()],
    model=InferenceClientModel(bill_to="VitalNest"),
    additional_authorized_imports=["os", "requests", "bs4", "pil", "base64", "io"],
    max_steps=1,
    # planning_interval=2,
    verbosity_level=0,
    name="insurance_agent",
    description="You answer insurance questions based on the InsuranceInfoRetriever tool.",
)

In [13]:
insurance_agent.prompt_templates['system_prompt']

'You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \'Thought:\', \'Code:\', and \'Observation:\' sequences.\n\nAt each step, in the \'Thought:\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\nThen in the \'Code:\' sequence, you should write the code in simple Python. The code sequence must end with \'<end_code>\' sequence.\nDuring each intermediate step, you can use \'print()\' to save whatever important information you will then need.\nThese print outputs will then appear in the \'Observation:\' field, which will be available as input for the next step.\nIn the end you have to return a final answer using the `final_answer` tool.\n\nHere

In [28]:
query = "what critical illnesses are covered under secure 4 in 1?"
# search_res = [
#     (
#         "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_42.png"
#     ),
#     (
#         "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_12.png"
#     ),
#     (
#         "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_1.png"
#     ),
# ]
insurance_agent.run(task=query
                    # f"""
# For the insurance search results below, answer the following query:{query}.
# """
)

INFO:src.insurance_assistants.complex_rag:Searching for 1 queries
INFO:src.insurance_assistants.complex_rag:Searching for query: Secure 4 in 1 critical illnesses
INFO:src.insurance_assistants.complex_rag:Processing 1 texts
INFO:src.insurance_assistants.complex_rag:Search result: [(np.float32(14.544364), 468), (np.float32(14.452676), 34), (np.float32(14.385986), 262), (np.float32(14.366423), 286)] for query: Secure 4 in 1 critical illnesses
INFO:src.insurance_assistants.complex_rag:✅ Retrieved the images for answering query.


Reached max steps.

'It seems there was an issue retrieving the specific details about the Secure 4 in 1 insurance plan. However, based on the naming convention and typical insurance plans, a "Secure 4 in 1" policy usually covers a selected number of critical illnesses. These can vary by insurer, but generally, they include:\n\n1. **Cancer** - Often includes various types and stages of cancer.\n2. **Heart Attack** - Usually covers acute myocardial infarction.\n3. **Stroke** - Typically covers ischemic or hemorrhagic stroke.\n4. **End Stage Renal Disease** - This commonly includes conditions that require sustained renal dialysis therapy.\n\nIt\'s important to check the specific terms and conditions of the Secure 4 in 1 policy document provided by your insurance company to confirm the exact list and specifics of coverage. You might also want to contact your insurer directly for an accurate and detailed breakdown.'

In [2]:
from smolagents import ToolCallingAgent

In [9]:
websearch_agent = ToolCallingAgent(
    model=InferenceClientModel(
        model_id="Qwen/Qwen3-30B-A3B", bill_to="VitalNest", temperature=0.1
    ),
    tools=[
        VisitWebpageTool(max_output_length=20_000),
        DuckDuckGoSearchTool(max_results=5),
    ],
    max_steps=4,
    verbosity_level=0,
    name="web_search_agent",
    planning_interval=2,
    description="Searches the web with a particular query.",
)

In [ ]:
# websearch_agent.run("what's multi head latent attention?")

In [11]:
wikipedia_agent = ToolCallingAgent(
    model=InferenceClientModel(model_id="Qwen/Qwen3-30B-A3B", bill_to="VitalNest", temperature=.1),
    tools=[WikipediaSearchTool(user_agent="WikiAssistant (merlin@example.com)")],
    max_steps=3,
    verbosity_level=0,
    name="wikipedia_agent",
    description="Searches Wikipedia for a topic.",
)

In [34]:
manager_agent = CodeAgent(
    tools=[PythonInterpreterTool(), FinalAnswerTool()],
    additional_authorized_imports=["os", "requests", "bs4"],
    model=InferenceClientModel(
        model_id="Qwen/Qwen3-30B-A3B", bill_to="VitalNest", temperature=0.1
    ),  # "Qwen/Qwen3-235B-A22B"
    managed_agents=[websearch_agent, wikipedia_agent, insurance_agent],
    max_steps=10,
    planning_interval=3,
    verbosity_level=0,
)

In [ ]:
manager_agent.run(
    task=f"""
For the insurance search results below, answer the following query:{query}.
search_res: {search_res}
"""
    #     """Search for `topic`:multi head latent attention in wikipedia.
    #                   If there's no information about the same in Wikipedia, search the web
    #                   for it and return a summary of the findings along with the search result links.
    # """
)

In [7]:
from src.insurance_assistants.complex_rag import RAG

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
rag_app = RAG()
rag_app.vectordb_id = "policy_wordings"

In [9]:
query = "what critical illnesses are covered under optima restore?"
res = rag_app.search_documents(query)


INFO:src.insurance_assistants.complex_rag:Searching for 1 queries
INFO:src.insurance_assistants.complex_rag:Searching for query: what critical illnesses are covered under optima restore?
INFO:src.insurance_assistants.complex_rag:Processing 1 texts
INFO:src.insurance_assistants.complex_rag:Search result: [(np.float32(17.525904), 376), (np.float32(17.395565), 377), (np.float32(17.221403), 378), (np.float32(16.622517), 346)] for query: what critical illnesses are covered under optima restore?
INFO:src.insurance_assistants.complex_rag:✅ Retrieved the images for answering query.


In [1]:
import torch
from PIL import Image
from transformers.utils.import_utils import is_flash_attn_2_available

from colpali_engine.models import ColQwen2_5, ColQwen2_5_Processor


In [ ]:

model = ColQwen2_5.from_pretrained(
    pretrained_model_name_or_path="vidore/colqwen2.5-v0.2",
    torch_dtype=torch.bfloat16,
    device_map="cuda:0",  # or "mps" if on Apple Silicon
    attn_implementation="flash_attention_2" if is_flash_attn_2_available() else None,
).eval()
processor = ColQwen2_5_Processor.from_pretrained("vidore/colqwen2.5-v0.2", use_fast=True)


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:

# Your inputs
images = [
    Image.new("RGB", (128, 128), color="white"),
    Image.new("RGB", (128, 128), color="black"),
]
queries = [
    "Is attention really all you need?",
    "What is the amount of bananas farmed in Salvador?",
]

# Process the inputs
batch_images = processor.process_images(images).to(model.device)
batch_queries = processor.process_queries(queries).to(model.device)

# Forward pass
with torch.no_grad():
    image_embeddings = model(**batch_images)
    query_embeddings = model(**batch_queries)

scores = processor.score_multi_vector(query_embeddings, image_embeddings)
